In [ ]:
from synrxn import DataLoader
from pathlib import Path
dl = DataLoader(task="classification", source="zenodo", version="0.0.6",
                cache_dir=Path("~/.cache/synrxn").expanduser())
print(dl.available_names())
df = dl.load('schneider_b')  
df.head()

In [ ]:
from synrxn import DataLoader
from pathlib import Path
dl = DataLoader(task="classification", source="zenodo", version="0.0.6",
                cache_dir=Path("~/.cache/synrxn").expanduser())
print(dl.available_names())
df = dl.load('schneider_b')  
df.head()

In [ ]:
from synrxn.data import DataLoader
from pathlib import Path
dl = DataLoader(task="class", source="github", version="v0.0.6",
                cache_dir=Path("~/.cache/synrxn").expanduser(),gh_enable=True)
print(dl.available_names())
df = dl.load('schneider_b')  
df.head()

In [ ]:
from synrxn.data import DataLoader
from pathlib import Path
dl = DataLoader(task="classification", source="commit", version="3e1612e2199e8b0e369fce3ed9aff3dda68e4c32" ,
                cache_dir=Path("~/.cache/synrxn").expanduser(),gh_enable=True)
print(dl.available_names())
df = dl.load('schneider_b')  
df.head()

In [ ]:
from synrxn.data import DataLoader
from pathlib import Path
dl = DataLoader(task="classification", source="commit", version="latest" ,
                cache_dir=Path("~/.cache/synrxn").expanduser(),gh_enable=True)
print(dl.available_names())
df = dl.load('schneider_b')  
df.head()

In [ ]:
from synrxn.split.repeated_kfold import RepeatedKFoldsSplitter
from synrxn.data import DataLoader
from pathlib import Path
dl = DataLoader(task="prop", source="commit", version="latest" ,
                cache_dir=Path("~/.cache/synrxn").expanduser(),gh_enable=True)
df = dl.load("b97xd3")
splitter = RepeatedKFoldsSplitter(n_splits=5, n_repeats=2, ratio=(8,1,1), shuffle=True, random_state=1)
splitter.prepare_splits(df, stratify=None)
train_df, val_df, test_df = splitter.get_split(repeat=1, fold=1, as_frame=True)
train_df.head(2)
